In [1]:
# Broker_vendor.py
import pandas as pd
import xml.etree.ElementTree as ET
import psycopg2
from datetime import datetime
import re
import cx_Oracle


# XML file path
xml_file_path = 'D:\\W\\RELEX\\RELEX\\Broker and Vendor\\Inbound Files\\VENDOR.xml'
# xml_file_path = 'D:\\W\\RELEX\\RELEX\\Broker and Vendor\\Inbound Files\\BROKER.xml'


# Open the XML file
with open(xml_file_path, 'r', encoding='utf-8') as file:
    xml_data = file.read()

# Database connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

# Replace these values with your Oracle database connection details
username = 'apps'
password = 'apps'
host = 'csebsd2db.cswg.com' #EBSD2
port = '1521'
service_name = 'csebsd2_int' #EBSD2

# Create a connection
dsn = cx_Oracle.makedsn(host, port, service_name=service_name)
connection = cx_Oracle.connect(username, password, dsn)

# Table name
table_name = "cspoms.XXPO_ORCL2CSPOMS_VEND_STG".upper()
with psycopg2.connect(**db_params) as post_conn:
    with post_conn.cursor() as post_cur:
        # Get the nextval for request_id
        nextval_request_id = "SELECT nextval('cspoms.xxpo_cs_relex_run_id_seq')"
        post_cur.execute(nextval_request_id)
        request_id = post_cur.fetchone()[0]
        post_conn.commit()
        print(f"Request ID: {request_id}")

        # Get the current timestamp
        current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"Current Timestamp: {current_timestamp}")
# Split the XML data into lines
lines = xml_data.splitlines()

# Regular expression pattern to match the elements
pattern = re.compile(r"<([^>]+)>([^<]+)</\1>")
tag_to_column = {
    'BICEPSVENDOR': 'BIC_VENDOR_NUMBER',
    'BIC_VENDOR_FACILITY':'BIC_VENDOR_FACILITY',
    'BUYERNUMBER': 'BIC_BUYER_NUMBER',
    'CRPLEVEL': 'BIC_CRP_LEVEL',
    'STATUS': 'BIC_STATUS',
    'PORECOMMENDEDIND': 'BIC_PO_RECOMMENDED_IND',
    'AFETYPE1': 'BIC_AFE_TYPE1',
    'AFEFAXNUMBER1': 'BIC_AFE_FAX_NUMBER',
    'AFECONTACT1': 'BIC_AFE_CONTACT',
    'MINIMUMQUANTITY': 'BIC_MINIMUM_QUANTITY',
    'MINIMUMTYPE': 'BIC_MINIMUM_TYPE',
    'BKTQUANTITY1': 'BIC_BRACKET_QUANTITY',
    'CURRENTBKTNUMBER': 'BIC_CURRENT_BKT_NUMBER',
    'CURRENTBKTQUANTITY': 'BIC_CURRENT_BKT_QUANTITY',
    'CURRENTBKTTYPE': 'BIC_CURRENT_BKT_TYPE',
    'MAXIMUMQUANTITY': 'BIC_MAXIMUM_QUANTITY',
    'MAXIMUMTYPE': 'BIC_MAXIMUM_TYPE',
    'ORDERINTERVALWEEKS': 'BIC_ORDER_INTERVAL_WEEKS',
    'LEADTIMESTATEDWEEKS': 'BIC_LEAD_TIME_STATED_WEEKS',
    'FIXEDREVIEWDAY1': 'BIC_FIXED_REVIEW_DAY1',
    'FIXEDREVIEWDAY2': 'BIC_FIXED_REVIEW_DAY2',
    'FIXEDREVIEWDAY3': 'BIC_FIXED_REVIEW_DAY3',
    'FIXEDREVIEWDAY4': 'BIC_FIXED_REVIEW_DAY4',
    'FIXEDREVIEWDAY5': 'BIC_FIXED_REVIEW_DAY5',
    'FIXEDREVIEWDAY6': 'BIC_FIXED_REVIEW_DAY6',
    'FIXEDREVIEWDAY7': 'BIC_FIXED_REVIEW_DAY7',
    'TARGETSERVICELEVEL': 'BIC_TARGET_SERVICE_LEVEL',
    'NAME': 'BIC_NAME',
    'ADDRESS1': 'BIC_ADDRESS_1',
    'CITY': 'BIC_CITY',
    'STATE': 'BIC_STATE',
    'ZIP': 'BIC_ZIP',
    'PHONE': 'BIC_PHONE',
    'CONTACT': 'BIC_CONTACT',
    'TRANSFERMATCHTYPE': 'BIC_TRANSFER_MATCH_TYPE',
    'PAYABLEVENDORNBR': 'BIC_PAYABLE_VENDOR_NBR',
    'LEADTIMEWEEKSTYPE': 'BIC_LEAD_TIME_WEEKS_TYPE',
    'DUNSREMITNO': 'BIC_DUNS_REMIT_NO',
    'DUNSREMITSUFFIX': 'BIC_DUNS_REMIT_SUFFIX',
    'FACILITYSHIPTO': 'BIC_FACILITY_SHIP_TO',
    'PRESCHED': 'BIC_PRE_SCHED_FLAG',
    'BROKERNUMBER': 'BIC_BROKER_NUMBER',
    'FREIGHTALLOW': 'BIC_FREIGHT_ALLOW',
    'FREIGHTALLOWTYPE': 'BIC_FREIGHT_ALLOW_TYPE',
    'TERMSPERCENT': 'BIC_TERMS_PERCENT',
    'TERMSDAYS': 'BIC_TERMS_DAYS',
    'TERMSNETDAYS': 'BIC_TERMS_NET_DAYS',
    'TERMSBASE': 'BIC_TERMS_BASE',
    'BILLFORSHORTEDDEALSFLAG': 'BIC_BILL_FOR_SHORT_DEALS_FLAG',
    'TRANSITDAYS': 'BIC_TRANSIT_DAYS',
    'DUNSNO': 'BIC_DUNS_NO',
    '2NDMINQUANTITY': 'BIC_2ND_MIN_QUANTITY',
    '2NDMINTYPE': 'BIC_2ND_MIN_TYPE',
    '2NDMAXQUANTITY': 'BIC_2ND_MAX_QUANTITY',
    '2NDMAXTYPE': 'BIC_2ND_MAX_TYPE',
    'FLAGPREPAID': 'BIC_FLAG_PREPAID',
    'FLAGPREPAYANDADD': 'BIC_FLAG_PREPAY_AND_ADD',
    'FLAGFREIGHTBILL': 'BIC_FLAG_FREIGHT_BILL',
    'FLAGBACKHAUL': 'BIC_FLAG_BACKHAUL',
    'FLAGTRUCK': 'BIC_FLAG_TRUCK',
    'FLAGRAIL': 'BIC_FLAG_RAIL',
    'BACKHAUL': 'BIC_BACKHAUL',
    'BACKHAULTYPE': 'BIC_BACKHAUL_TYPE',
    'FREIGHTBILL': 'BIC_FREIGHTBILL',
    'ORDERSEQUENCE': 'BIC_ORDER_SEQUENCE',
    'SSAALLOWPERCENT': 'BIC_SSA_ALLOW_PERCENT',
    'SSAALLOWBASIS': 'BIC_SSA_ALLOW_BASIS',
    'SSAALLOWACCOUNT': 'BIC_SSA_ALLOW_ACCOUNT',
    'TRANSACTIONID': 'BIC_TRANSACTION_ID',
    'COUNTRY': 'BIC_COUNTRY',
    'ORDERFILTER': 'BIC_FILTER_VENDOR_FLAG',
    'LBLIMIT': 'BIC_LOAD_BLDG_LIMIT_FLAG',
    'WAREHOUSECODE': 'BIC_WAREHOUSE_CODE',
    'FLAGEXTBCKHL': 'BIC_EXTERNAL_BACKHAUL'
}
# print('tag_to_column : ',len(tag_to_column))
# Loop through each line in the XML data
for line in lines:
    # Skip empty lines or lines containing only whitespace
    if not line.strip():
        continue

    # Define a dictionary to store values
    element_values = {}

    # Find all matches in the text data
    matches = re.findall(pattern, line)

    # Store the matches in the dictionary
    for tag, value in matches:
        element_values[tag] = value.strip()
    # Assuming 'BICEPSVENDOR' is present in element_values
    if 'BICEPSVENDOR' in element_values:
        bicepsvendor_value = element_values['BICEPSVENDOR']

        # Check if the value has at least 3 characters
        if len(bicepsvendor_value) >= 3:
            # Extract the first 3 digits
            facility_value = bicepsvendor_value[:3]

            # Assign the first 3 digits to a new key
            element_values['BIC_VENDOR_FACILITY'] = facility_value

            # Update 'BICEPSVENDOR' with the remaining value
            element_values['BICEPSVENDOR'] = bicepsvendor_value[3:]
#     print('element_values:############',element_values)
    # Now 'BIC_VENDOR_FACILITY' and updated 'BICEPSVENDOR' are present in element_values
    # Replace column names with the ones from tag_to_column
    mapped_values = {tag_to_column[key]: value for key, value in element_values.items() if key in tag_to_column}
    # Include additional columns in the mapping
    mapped_values['REQUEST_ID'] = request_id
    mapped_values['CREATION_DATE'] = current_timestamp
    mapped_values['LAST_UPDATE_DATE'] = current_timestamp
    mapped_values['CREATED_BY'] = -1
    mapped_values['LAST_UPDATED_BY'] = -1
    print('mapped_values:########0',mapped_values)
    # Generate SQL INSERT statement
    columns_str = ",".join(mapped_values.keys())
    values_str = ",".join(f"'{v}'" for v in mapped_values.values())

    insert_query = f"INSERT INTO {table_name} ({columns_str}) VALUES ({values_str});"
#     print(insert_query)

# Establish a database connection and execute the INSERT statement
    with psycopg2.connect(**db_params) as conn:
        with conn.cursor() as cursor:
            cursor.execute(insert_query)
            print(insert_query)


        print(f"Record inserted: {mapped_values}")

# Establish a database connection and execute the INSERT statement
with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cursor:
    # Execute the SQL statement
        
        # Execute the SQL statement to select all rows from the table
        cursor.execute(f"SELECT * FROM XXPO_ORCL2CSPOMS_VEND_STG where PROCESS_STATUS ='N'")

        # Fetch all rows into a list of tuples
        result_rows = cursor.fetchall()

        # Get the column names from the cursor description
        column_names = [desc[0] for desc in cursor.description]

        # Create a DataFrame using the result and column names
        stg_df = pd.DataFrame(result_rows, columns=column_names)

        # Print or use the DataFrame as needed
        print(stg_df)
        # Extract the 'bic_vendor_number' column and convert it to a list
        bic_vendor_numbers_list = stg_df['bic_vendor_number'].tolist()

        # Convert the list of strings to a list of integers
        bic_vendor_numbers_list = list(map(int, bic_vendor_numbers_list))

        # Print or use the list as needed
        print(bic_vendor_numbers_list)

conn = psycopg2.connect(**db_params)
# Create a cursor object to execute SQL queries
cur = conn.cursor()
# Print or use the new value as needed
# print("New CS_VENDOR_ID:", new_cs_vendor_id)

# Convert the list of numbers to a string with quotes for the SQL query
vendor_numbers_str = ', '.join([f"'{num}'" for num in bic_vendor_numbers_list])


# Construct the SQL query
sql_query = f"SELECT * FROM APPS.XXAP_VENDOR_MASTER WHERE VENDOR_NBR IN ({vendor_numbers_str})"

oracle_df = pd.read_sql(sql_query, connection)
oracle_df['VENDOR_NBR'] = oracle_df['VENDOR_NBR'].str.zfill(6)
# Display the DataFrame
print(oracle_df.head())

# Merge oracle_df with stg_df based on the common column 'bic_vendor_number' and 'VENDOR_NBR'
combined_df = pd.merge(stg_df, oracle_df, how='left', left_on='bic_vendor_number', right_on='VENDOR_NBR')

#############################################################################
def clone_rows_active_vendor_facility(input_df, column_to_change, new_values):
    """
    Clone rows in a DataFrame with Facility changed based on a list of values.
    Skip cloning if the new value matches the original row's condition.
    Change the 'VENDOR_TYPE' column to 'C' in the resulting DataFrame.

    Parameters:
    - input_df (pd.DataFrame): The input DataFrame.
    - column_to_change (str): The name of the column to be changed.
    - new_values (list): The list of new values for the specified column.

    Returns:
    - pd.DataFrame: The resulting DataFrame with cloned rows and the specified column changed.
    """
    new_rows = pd.DataFrame()

    for row in input_df.iterrows():
        bic_vendor_facility_value = row[column_to_change]
        # Clone rows only if the new value is different from the original value
        for new_value in new_values:
            if new_value != bic_vendor_facility_value:
                new_row = row.copy()
                new_row[column_to_change] = new_value
                new_rows = new_rows.append(new_row, ignore_index=True)

    # Change the 'VENDOR_TYPE' column to 'C' in the resulting DataFrame
    new_rows['VENDOR_TYPE'] = 'C'

    return new_rows


#############################################################################

#############################################################################
# Step 1: Retrieve the last_run from PostgreSQL
# cur.execute("SELECT profile_value FROM cspoms_common_profiles WHERE profile_name = 'I1619_CSPOMS_RLX_BROKER_VENDOR'")
# result = cur.fetchone()
# print(f'TIme in the Profile Value',result)

# Retrieve the current timestamp from the database
# profile_update_query = "SELECT CURRENT_TIMESTAMP"
# cur.execute(profile_update_query)
# current_timestamp = cur.fetchone()[0]
# print(f'Time it is going to update next',current_timestamp)
# Convert the retrieved value to a datetime object
# current_timestamp_dt = datetime.strptime(str(current_timestamp), "%Y-%m-%d %H:%M:%S.%f%z")

# Format the datetime object to match the desired format
# current_timestamp_str = current_timestamp_dt.strftime("%Y-%m-%d %H:%M:%S")

# if result:
    # Step 2: Convert the retrieved value to a datetime object
    # last_run_str = result[0]  # Assuming the result is a string
    # Remove the UTC offset before parsing
    # Remove the UTC offset and microseconds before parsing
    # last_run_str_without_offset = last_run_str.split('+')[0].split('.')[0]
    # last_run_datetime = datetime.strptime(last_run_str_without_offset, "%Y-%m-%d %H:%M:%S")
    # Step 3: Format the datetime object to match the Oracle date format
    # last_run_formatted = last_run_datetime.strftime("%Y-%m-%d %H:%M:%S")

    # Step 4: Use the formatted date in the Oracle SQL query
    # delta_run_query = f"SELECT * FROM XXAP_VENDOR_MASTER WHERE last_update_date > TO_DATE('{last_run_formatted}', 'YYYY-MM-DD HH24:MI:SS')"
    # delta_run_query = f"SELECT * FROM XXAP_VENDOR_MASTER  WHERE ROWNUM <1000"
    # delta_df = pd.read_sql(delta_run_query, connection)
    # print('@@@@@@@delta_df', delta_df.columns)
    # if not delta_df.empty:
        # print('@@@@@@@@@@@@@combined_df,',combined_df.columns)
        # delta_df['VENDOR_NBR'] = oracle_df['VENDOR_NBR'].str.zfill(6)
        # Merge delta_df with combined_df based on 'VENDOR_NBR'
        # final_df = pd.merge(combined_df, delta_df, on='VENDOR_NBR', how='outer')
        # print('final_df', final_df)
        # final_df = pd.merge(combined_df, delta_df, how='left', left_on='bic_vendor_number', right_on='VENDOR_NBR')
    # else:
        # print('No rows to fetch in delta logic from the last run')
final_df = combined_df
        # print('final_df', final_df)
    
# else:
#     print("No Profile value in cspoms_common_profiles for I1619_CSPOMS_RLX_BROKER_VENDOR. taking full records")
#     oracle_query = f"SELECT * FROM XXAP_VENDOR_MASTER WHERE ROWNUM <10"



# Display the merged DataFrame
print(final_df)
# Define a custom function to determine LOAD_BUILDING
def determine_load_building(row):
    try:
        if (
            row['DIV_VNDR_FLG'] is not None and not pd.isna(row['DIV_VNDR_FLG']) or
            row['VENDOR_FOODLINK_FLG'] is not None and not pd.isna(row['VENDOR_FOODLINK_FLG']) or
            row['VENDOR_FREIGHT_FLG'] is not None and not pd.isna(row['VENDOR_FREIGHT_FLG'])
        ):
            return 'F'
        else:
            return 'B'
    except Exception as e:
        print(f"Error in determine_load_building function: {e}")
        print(f"Row details: {row}")
        raise  # Reraise the exception to see the traceback

# Apply the custom function to create the LOAD_BUILDING column
final_df['LOAD_BUILDING'] =  final_df.apply(determine_load_building, axis=1)
# Update 'VENDOR_TYPE' column based on conditions
final_df.loc[ final_df['VENDOR_TYPE'] == 'Broker', 'VENDOR_TYPE'] = 'B'
final_df.loc[final_df['VENDOR_TYPE'] == 'Buyer Vendor', 'VENDOR_TYPE'] = 'V'

# Duplicate the 'SHIP_TO_LOCATION_NAME' column as 'FACILITY_NAME'
final_df = final_df.assign(FACILITY_NAME=final_df['SHIP_TO_LOCATION_NAME'])
# CREATION_DATE	 :  	PREPAYADD_BUY_RATE_CHANGED
final_df = final_df.assign(PREPAYADD_BUY_RATE_CHANGED= final_df['CREATION_DATE'])
final_df = final_df.assign(BACKHAUL_BUY_RATE_CHANGED= final_df['CREATION_DATE'])
final_df = final_df.assign(FREIGHT_BILL_BUY_RATE_CHANGED= final_df['CREATION_DATE'])
# BIC_SSA_FLAG	 : 	SSA_RATE_BASIS_GROSS_FLAG
final_df = final_df.assign(SSA_RATE_BASIS_GROSS_FLAG= final_df['bic_ssa_flag'])
final_df = final_df.assign(MAXIMUM_BKT_TYPE= final_df['VENDOR_ORDER_UNIT'])
#     'VENDOR_ORDER2_UNIT':'MAX_BKT_TYPE_2ND'
final_df = final_df.assign(MAX_BKT_TYPE_2ND= final_df['VENDOR_ORDER2_UNIT'])
# 	,'bic_backhaul_type'	 : 	'BACKHAUL_BUY_RATE_TYPE'
final_df = final_df.assign(BACKHAUL_BUY_RATE_TYPE= final_df['bic_backhaul_type'])
#     ,'bic_flag_prepay_and_add'	 : 	'PRE_PAY_ADD_FLAG'
final_df = final_df.assign(PRE_PAY_ADD_FLAG= final_df['bic_flag_prepay_and_add'])

############################################################
#Drop the column before renaming
############################################################
# List of columns to drop (including one that doesn't exist)
columns_to_drop = ['OTHER_COLUMN_1', 'OTHER_COLUMN_2', 'NON_EXISTENT_COLUMN','VENDOR_FACILITY']

# Drop the specified columns with errors='ignore'
final_df = final_df.drop(columns=columns_to_drop, errors='ignore')



############################################################


# # Rename columns
merged_df = final_df.rename(columns={
    'VENDOR_SITE_ID': 'CS_VEND_SITE_ID',
    'VENDOR_NBR': 'VENDOR_NUMBER',
    'BUYER_VENDOR_NAME': 'VENDOR_NAME',
    'bic_vendor_facility': 'VENDOR_FACILITY',
    'BICEPS_BUYER_CODE': 'BUYER_ID',
    'bic_warehouse_code': 'WAREHOUSE_CODE',
    'VENDOR_FOODLINK_FLG': 'FOOD_LINK_FLAG',
    'VENDOR_BROKER_NBR': 'BROKER_NUMBER',
    'VENDOR_BROKER_NAME': 'BROKER_NAME',
    'bic_flag_truck': 'TRUCK_FLAG',
    'bic_flag_rail': 'RAIL_FLAG',
    'bic_afe_contact': 'PO_TRANSMISSION_TYPE',  
    'bic_afe_fax_number': 'PO_TRANSMISSION_VALUE',
    'bic_freight_allow': 'FREIGHT_ALLOW_BUY_RATE',
    'LEAD_TIME_CALENDAR_IND' : 'LEAD_TIME_STATED_TYPE',
    'VENDOR_LEAD_DAYS' : 'LEAD_TIME_STATED_DAYS',
    'VENDOR_MIN_ORDER_SIZE' : 'MINIMUM_BKT_0',
    'VENDOR_ORDER_UNIT' : 'MINIMUM_BKT_0_TYPE',
    'VENDOR_MAX_ORDER_SIZE' : 'MAXIMUM_BKT',
    'VENDOR_MIN_ORDER2_SIZE':'MIN_BKT_2ND',
    'VENDOR_ORDER2_UNIT':'MIN_BKT_TYPE_2ND',
    'VENDOR_MAX_ORDER2_SIZE':'MAX_BKT_2ND'
    ,'VENDOR_DLVR_MONDAY_FLG' :  'FIXED_REVIEW_MON'
    ,'VENDOR_DLVR_TUESDAY_FLG' :  'FIXED_REVIEW_TUE'
    ,'VENDOR_DLVR_WEDNESDAY_FLG' :  'FIXED_REVIEW_WED'
    ,'VENDOR_DLVR_THURSDAY_FLG' :  'FIXED_REVIEW_THU'
    ,'VENDOR_DLVR_FRIDAY_FLG' :  'FIXED_REVIEW_FRI'
    ,'VENDOR_DLVR_SATURDAY_FLG' :  'FIXED_REVIEW_SAT'
    ,'VENDOR_DLVR_SUNDAY_FLG' :  'FIXED_REVIEW_SUN'
    ,'bic_order_sequence' : 'ORDER_SEQUENCE'
    ,'FILTER_VENDOR_FLG' : 'ORDER_FILTER_FLAG'
    ,'bic_freight_allow'	 :  	'FREIGHT_ALLOW_BUY_RATE'
    ,'bic_freight_allow_type'	 :  	'FREIGHT_ALLOW_BUY_RATE_TYPE'
    ,'creation_date'	 :  	'FREIGHT_ALLOW_BUY_RATE_CHANGED'
    ,'bic_backhaul_type'	 :  	'PREPAYADD_BUY_RATE_TYPE'
    ,'bic_freightbill'	 : 'FREIGHT_BILL_BUY_RATE'
	,'bic_freightbill_type'	 : 	'FREIGHT_BILL_BUY_RATE_TYPE'
	,'bic_backhaul'	 : 	'BACKHAUL_BUY_RATE'
    ,'bic_ssa_flag'	 : 	'SSA_RATE_BASIS_NET_FLAG'
    ,'bic_flag_prepaid'	 : 	'PRE_PAY_FLAG'
    ,'bic_flag_freight_bill'	 : 	'FREIGHT_BILL_FLAG'
    ,'bic_flag_backhaul'	 : 	'BACKHAUL_FLAG'
    ,'bic_external_backhaul'	 : 	'EXTERNAL_BACKHAUL_FLAG'
    ,'bic_ssa_allow_percent'	 : 	'SSA_RATE'
    ,'bic_ssa_allow_basis'	 : 	'SSA_RATE_TYPE'
    ,'bic_ssa_allow_account'	 : 	'SSA_RATE_BASIS_ACC'
    ,'VENDOR_DISC_PCT'	 : 	'CASH_DISC_TERMS'
    ,'VENDOR_DISCOUNT_DAYS_NBR'	 : 	'CASH_DISC_TERMS_DAYS'
    ,'VENDOR_NET_DAYS'	 : 	'CASH_DISC_TERMS_NET'

   
})
#     ,'bic_ssa_flag'	 : 	'SSA_RATE_BASIS_NET_FLAG'
#     ,'bic_flag_prepay_and_add'	 :  	'PREPAYADD_BUY_RATE'
# Display the resulting DataFrame
# print(merged_df)

############################################################
######## CLONE LOGIC TO COPY TO ALL ACTIVE ORGS
print('# CLONE LOGIC TO COPY TO ALL ACTIVE ORGS')
############################################################
# Condition 1: VENDOR_FOODLINK_FLG is 1 or 2
condition1 = (merged_df['FOOD_LINK_FLAG'] == '1') | (merged_df['FOOD_LINK_FLAG'] == '2')

# Condition 2: VENDOR_FREIGHT_FLG is 'Y'
condition2 = merged_df['VENDOR_FREIGHT_FLG'] == 'Y'

# Condition 3: VENDOR_HOME_LOC_DEST is 21
condition3 = merged_df['VENDOR_FACILITY'] == '21'

# Combine the conditions using OR (|) as per your logic
final_condition = condition1 | condition2 | condition3

# Apply the final condition to filter the rows
filtered_rows = merged_df[final_condition].copy()

# Display the filtered rows
print(filtered_rows)

# Now you can create input_df from filtered_rows and pass it to your function
input_df = filtered_rows.copy()
print(f'input_df that should be clone for all orgs',input_df)
all_active_orgs = "SELECT facility_code FROM cspoms_facility_master WHERE inactive_date IS NULL"
cur.execute(all_active_orgs)

# Fetch all results into a list
all_active_orgs_list = cur.fetchall()

# Extract facility_code values from the result list
facility_codes_list = [row[0] for row in all_active_orgs_list]


############################################################
def clone_rows_active_vendor_facilty(input_df, column_to_change, new_values):
    """
    Clone rows in a DataFrame with Facility changed based on a list of values.
    Skip cloning if the new value matches the original row's condition.
    Remove the corresponding cloned rows based on the original rows' condition.

    Parameters:
    - input_df (pd.DataFrame): The input DataFrame.
    - column_to_change (str): VENDOR_FACILITY
    - new_values (list): The list of facility_codes_list

    Returns:
    - pd.DataFrame: The resulting DataFrame with cloned rows and the specified column changed.
    """
    new_rows = pd.DataFrame()

    for  row in input_df.iterrows():
        bic_vendor_facility_value = row[column_to_change]
        print(bic_vendor_facility_value)
        # Clone rows only if the new value is different from the original value
        for new_value in new_values:
            if new_value != bic_vendor_facility_value:
                new_row = row.copy()
                new_row[column_to_change] = new_value
                new_rows = new_rows.append(new_row, ignore_index=True)

#     # Drop the corresponding cloned rows based on the original rows' condition
#     new_rows = new_rows.drop(index=rows_to_delete)

    return new_rows      
############################################################          
result_df = clone_rows_active_vendor_facilty(input_df, 'VENDOR_FACILITY', facility_codes_list)
print('@@@@@@@@@ result_df',result_df)            
# Concatenate result_df to merged_df
merged_df = pd.concat([merged_df, result_df], ignore_index=True)

# Display the updated merged_df
print('@@@@@@@@@ merged_df', merged_df)

print(merged_df)
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]





# Construct the DELTA LOGIC query
# sql_query = f"SELECT * FROM APPS.XXAP_VENDOR_MASTER WHERE LAST_UPDATE_DATE > ({vendor_numbers_str})"

# oracle_df = pd.read_sql(sql_query, connection)


# print(merged_df)



def upsert_vendor_master_and_childs(merged_df,cur):
    # Iterate over DataFrame rows for upsert
    for index, row in merged_df.iterrows():
        # cur.execute("SELECT nextval('cs_vendor_id_seq')")
        # next_sequence_value = cur.fetchone()[0]
        
        # print(next_sequence_value)
        cur.execute(
        f'''INSERT INTO cspoms.XXPO_VENDOR_MASTER (
            CS_VENDOR_ID,
            VENDOR_NUMBER,
            VENDOR_FACILITY,
            CS_VEND_SITE_ID,
            CONNECTING_VENDOR,
            VENDOR_NAME,
            VENDOR_TYPE,
            FACILITY_NAME,
            VENDOR_STATUS,
            AP_VENDOR_NBR,
            AP_VENDOR_NAME,
            BUYER_ID,
            BUYER_NAME,
            LOAD_BUILDING,
            WAREHOUSE_CODE,
            FOOD_LINK_FLAG,
            ADDR_LINE1,
            ADDR_LINE2,
            ADDR_LINE3,
            CITY,
            STATE,
            ZIP_CODE,
            COUNTRY,
            PHONE_NBR,
            VENDOR_CONTACT_NAME,
            SHIP_TO_LOCATION_NAME,
            BILL_TO_LOCATION_NAME,
            BROKER_NUMBER,
            BROKER_NAME,
            TRUCK_FLAG,
            RAIL_FLAG,
            PO_TRANSMISSION_TYPE,
            PO_TRANSMISSION_VALUE,
            REQUEST_ID,
            CREATION_DATE,
            LAST_UPDATE_DATE,
            CREATED_BY,
            LAST_UPDATED_BY
        ) 
        VALUES (
            
             nextval('cs_vendor_id_seq'),
            '{row['VENDOR_NUMBER']}'::numeric,
            '{row['VENDOR_FACILITY']}',
            '{row['CS_VEND_SITE_ID']}',
            '{row['CONNECTING_VENDOR']}',
            '{row['VENDOR_NAME']}',
            '{row['VENDOR_TYPE']}',
            '{row['FACILITY_NAME']}',
            '{row['VENDOR_STATUS']}',
            '{row['AP_VENDOR_NBR']}',
            '{row['AP_VENDOR_NAME']}',
            '{row['BUYER_ID']}',
            '{row['BUYER_NAME']}',
            '{row['LOAD_BUILDING']}',
            '{row['WAREHOUSE_CODE']}',
            '{row['FOOD_LINK_FLAG']}',
            '{row['ADDR_LINE1']}',
            '{row['ADDR_LINE2']}',
            '{row['ADDR_LINE3']}',
            '{row['CITY']}',
            '{row['STATE']}',
            '{row['ZIP_CODE']}',
            '{row['COUNTRY']}',
            '{row['PHONE_NBR']}',
            '{row['VENDOR_CONTACT_NAME']}',
            '{row['SHIP_TO_LOCATION_NAME']}',
            '{row['BILL_TO_LOCATION_NAME']}',
            '{row['BROKER_NUMBER']}',
            '{row['BROKER_NAME']}',
            '{row['TRUCK_FLAG']}',
            '{row['RAIL_FLAG']}',
            '{row['PO_TRANSMISSION_TYPE']}',
            '{row['PO_TRANSMISSION_VALUE']}',
            '{row['request_id']}',
            current_timestamp,
            current_timestamp,
            '-1',
            '-1'


        ) 
        ON CONFLICT (VENDOR_NUMBER, VENDOR_FACILITY) DO UPDATE SET 
            VENDOR_NUMBER = EXCLUDED.VENDOR_NUMBER,
            VENDOR_FACILITY = EXCLUDED.VENDOR_FACILITY,
            CS_VEND_SITE_ID = EXCLUDED.CS_VEND_SITE_ID,
            CONNECTING_VENDOR = EXCLUDED.CONNECTING_VENDOR,
            VENDOR_NAME = EXCLUDED.VENDOR_NAME,
            VENDOR_TYPE = EXCLUDED.VENDOR_TYPE,
            FACILITY_NAME = EXCLUDED.FACILITY_NAME,
            VENDOR_STATUS = EXCLUDED.VENDOR_STATUS,
            AP_VENDOR_NBR = EXCLUDED.AP_VENDOR_NBR,
            AP_VENDOR_NAME = EXCLUDED.AP_VENDOR_NAME,
            BUYER_ID = EXCLUDED.BUYER_ID,
            BUYER_NAME = EXCLUDED.BUYER_NAME,
            LOAD_BUILDING = EXCLUDED.LOAD_BUILDING,
            WAREHOUSE_CODE = EXCLUDED.WAREHOUSE_CODE,
            FOOD_LINK_FLAG = EXCLUDED.FOOD_LINK_FLAG,
            ADDR_LINE1 = EXCLUDED.ADDR_LINE1,
            ADDR_LINE2 = EXCLUDED.ADDR_LINE2,
            ADDR_LINE3 = EXCLUDED.ADDR_LINE3,
            CITY = EXCLUDED.CITY,
            STATE = EXCLUDED.STATE,
            ZIP_CODE = EXCLUDED.ZIP_CODE,
            COUNTRY = EXCLUDED.COUNTRY,
            PHONE_NBR = EXCLUDED.PHONE_NBR,
            VENDOR_CONTACT_NAME = EXCLUDED.VENDOR_CONTACT_NAME,
            SHIP_TO_LOCATION_NAME = EXCLUDED.SHIP_TO_LOCATION_NAME,
            BILL_TO_LOCATION_NAME = EXCLUDED.BILL_TO_LOCATION_NAME,
            BROKER_NUMBER = EXCLUDED.BROKER_NUMBER,
            BROKER_NAME = EXCLUDED.BROKER_NAME,
            TRUCK_FLAG = EXCLUDED.TRUCK_FLAG,
            RAIL_FLAG = EXCLUDED.RAIL_FLAG,
            PO_TRANSMISSION_TYPE = EXCLUDED.PO_TRANSMISSION_TYPE,
            PO_TRANSMISSION_VALUE = EXCLUDED.PO_TRANSMISSION_VALUE,
            REQUEST_ID = EXCLUDED.REQUEST_ID,
            CREATION_DATE = EXCLUDED.CREATION_DATE,
            LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
            CREATED_BY = EXCLUDED.CREATED_BY,
            LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY

            RETURNING CS_VENDOR_ID'''
    )


        # Fetch the generated C1 value
        generated_c1 = cur.fetchone()[0]

        # If C1 was generated or updated, insert/update other tables
        if generated_c1:
            print('insert if')
            cur.execute(
        f'''
        INSERT INTO XXPO_VENDOR_ORDER_POINT (
            CS_VENDOR_ID,
            MAIN_FACILITY,
            BUILD_TO_DAYS_SUPPLY,
            DAYS_TO_BUILD_TO,
            USE_CYCLE_STOCK_AS_MIN,
            USE_FAC_WHSE_SS,
            LEAD_TIME_OVRD_DAYS_1,
            LEAD_TIME_OVRD_DAYS_2,
            LEAD_TIME_OVRD_DAYS_3,
            LEAD_TIME_OVRD_DAYS_4,
            LEAD_TIME_OVRD_DAYS_5,
            LEAD_TIME_OVRD_DAYS_6,
            LEAD_TIME_OVRD_DAYS_7,
            APPLY_RATIONING,
            OP_DAYS_1ST_RUN_1,
            OP_DAYS_1ST_RUN_2,
            OP_DAYS_1ST_RUN_3,
            OP_DAYS_1ST_RUN_4,
            OP_DAYS_1ST_RUN_5,
            OP_DAYS_1ST_RUN_6,
            OP_DAYS_1ST_RUN_7,
            OP_DAYS_2ND_RUN_1,
            OP_DAYS_2ND_RUN_2,
            OP_DAYS_2ND_RUN_3,
            OP_DAYS_2ND_RUN_4,
            OP_DAYS_2ND_RUN_5,
            OP_DAYS_2ND_RUN_6,
            OP_DAYS_2ND_RUN_7,
            SHIP_WITH_VENDOR,
            CONNECTING_VENDOR,
            HAWAII_TRANS_METHOD,
            REQUEST_ID,
            CREATION_DATE,
            LAST_UPDATE_DATE,
            CREATED_BY,
            LAST_UPDATED_BY
        )
        VALUES (
            {generated_c1},
            '{row['MAIN_FACILITY']}',
            '{row['BUILD_TO_DAYS_SUPPLY']}',
            '{row['DAYS_TO_BUILD_TO']}',
            '{row['USE_CYCLE_STOCK_AS_MIN']}',
            '{row['USE_FAC_WHSE_SS']}',
            '{row['LEAD_TIME_OVRD_DAYS_1']}',
            '{row['LEAD_TIME_OVRD_DAYS_2']}',
            '{row['LEAD_TIME_OVRD_DAYS_3']}',
            '{row['LEAD_TIME_OVRD_DAYS_4']}',
            '{row['LEAD_TIME_OVRD_DAYS_5']}',
            '{row['LEAD_TIME_OVRD_DAYS_6']}',
            '{row['LEAD_TIME_OVRD_DAYS_7']}',
            '{row['APPLY_RATIONING']}',
            '{row['OP_DAYS_1ST_RUN_1']}',
            '{row['OP_DAYS_1ST_RUN_2']}',
            '{row['OP_DAYS_1ST_RUN_3']}',
            '{row['OP_DAYS_1ST_RUN_4']}',
            '{row['OP_DAYS_1ST_RUN_5']}',
            '{row['OP_DAYS_1ST_RUN_6']}',
            '{row['OP_DAYS_1ST_RUN_7']}',
            '{row['OP_DAYS_2ND_RUN_1']}',
            '{row['OP_DAYS_2ND_RUN_2']}',
            '{row['OP_DAYS_2ND_RUN_3']}',
            '{row['OP_DAYS_2ND_RUN_4']}',
            '{row['OP_DAYS_2ND_RUN_5']}',
            '{row['OP_DAYS_2ND_RUN_6']}',
            '{row['OP_DAYS_2ND_RUN_7']}',
            '{row['SHIP_WITH_VENDOR']}',
            '{row['CONNECTING_VENDOR']}',
            '{row['HAWAII_TRANS_METHOD']}',
            '{row['request_id']}',
            current_timestamp,
            current_timestamp,
            '-1',
            '-1'
        )
        ON CONFLICT (CS_VENDOR_ID) DO UPDATE SET
            MAIN_FACILITY = EXCLUDED.MAIN_FACILITY,
            BUILD_TO_DAYS_SUPPLY = EXCLUDED.BUILD_TO_DAYS_SUPPLY,
            DAYS_TO_BUILD_TO = EXCLUDED.DAYS_TO_BUILD_TO,
            USE_CYCLE_STOCK_AS_MIN = EXCLUDED.USE_CYCLE_STOCK_AS_MIN,
            USE_FAC_WHSE_SS = EXCLUDED.USE_FAC_WHSE_SS,
            LEAD_TIME_OVRD_DAYS_1 = EXCLUDED.LEAD_TIME_OVRD_DAYS_1,
            LEAD_TIME_OVRD_DAYS_2 = EXCLUDED.LEAD_TIME_OVRD_DAYS_2,
            LEAD_TIME_OVRD_DAYS_3 = EXCLUDED.LEAD_TIME_OVRD_DAYS_3,
            LEAD_TIME_OVRD_DAYS_4 = EXCLUDED.LEAD_TIME_OVRD_DAYS_4,
            LEAD_TIME_OVRD_DAYS_5 = EXCLUDED.LEAD_TIME_OVRD_DAYS_5,
            LEAD_TIME_OVRD_DAYS_6 = EXCLUDED.LEAD_TIME_OVRD_DAYS_6,
            LEAD_TIME_OVRD_DAYS_7 = EXCLUDED.LEAD_TIME_OVRD_DAYS_7,
            APPLY_RATIONING = EXCLUDED.APPLY_RATIONING,
            OP_DAYS_1ST_RUN_1 = EXCLUDED.OP_DAYS_1ST_RUN_1,
            OP_DAYS_1ST_RUN_2 = EXCLUDED.OP_DAYS_1ST_RUN_2,
            OP_DAYS_1ST_RUN_3 = EXCLUDED.OP_DAYS_1ST_RUN_3,
            OP_DAYS_1ST_RUN_4 = EXCLUDED.OP_DAYS_1ST_RUN_4,
            OP_DAYS_1ST_RUN_5 = EXCLUDED.OP_DAYS_1ST_RUN_5,
            OP_DAYS_1ST_RUN_6 = EXCLUDED.OP_DAYS_1ST_RUN_6,
            OP_DAYS_1ST_RUN_7 = EXCLUDED.OP_DAYS_1ST_RUN_7,
            OP_DAYS_2ND_RUN_1 = EXCLUDED.OP_DAYS_2ND_RUN_1,
            OP_DAYS_2ND_RUN_2 = EXCLUDED.OP_DAYS_2ND_RUN_2,
            OP_DAYS_2ND_RUN_3 = EXCLUDED.OP_DAYS_2ND_RUN_3,
            OP_DAYS_2ND_RUN_4 = EXCLUDED.OP_DAYS_2ND_RUN_4,
            OP_DAYS_2ND_RUN_5 = EXCLUDED.OP_DAYS_2ND_RUN_5,
            OP_DAYS_2ND_RUN_6 = EXCLUDED.OP_DAYS_2ND_RUN_6,
            OP_DAYS_2ND_RUN_7 = EXCLUDED.OP_DAYS_2ND_RUN_7,
            SHIP_WITH_VENDOR = EXCLUDED.SHIP_WITH_VENDOR,
            CONNECTING_VENDOR = EXCLUDED.CONNECTING_VENDOR,
            HAWAII_TRANS_METHOD = EXCLUDED.HAWAII_TRANS_METHOD,
            REQUEST_ID = EXCLUDED.REQUEST_ID,
            CREATION_DATE = EXCLUDED.CREATION_DATE,
            LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
            CREATED_BY = EXCLUDED.CREATED_BY,
            LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY
        '''
    )


            cur.execute(
                f'''
                INSERT INTO XXPO_VENDOR_ORDER_INFO (
                    CS_VENDOR_ID,
                    VENDOR_NUMBER,
                    CONNECTING_VENDOR,
                    TRANSIT_DAYS,
                    LAG_DAYS,
                    LEAD_TIME_STATED_TYPE,
                    LEAD_TIME_STATED_DAYS,
                    MINIMUM_BKT_0,
                    MINIMUM_BKT_0_TYPE,
                    MAXIMUM_BKT,
                    MAXIMUM_BKT_TYPE,
                    MIN_BKT_2ND,
                    MIN_BKT_TYPE_2ND,
                    MAX_BKT_2ND,
                    MAX_BKT_TYPE_2ND,
                    FIXED_REVIEW_MON,
                    FIXED_REVIEW_TUE,
                    FIXED_REVIEW_WED,
                    FIXED_REVIEW_THU,
                    FIXED_REVIEW_FRI,
                    FIXED_REVIEW_SAT,
                    FIXED_REVIEW_SUN,
                    VENDOR_SERVICE_LEVEL,
                    DAYS_TO_BUY_OVERRIDE,
                    ORDER_SEQUENCE,
                    LOAD_BLDG_LIMIT_FLG,
                    ORDER_FILTER_FLAG,
                    REQUEST_ID,
                    CREATION_DATE,
                    LAST_UPDATE_DATE,
                    CREATED_BY,
                    LAST_UPDATED_BY
                )
                VALUES (
                    {generated_c1},
                    '{row['VENDOR_NUMBER']}'::numeric,
                    '{row['CONNECTING_VENDOR']}',
                    '{row['TRANSIT_DAYS']}',
                    '{row['LAG_DAYS']}',
                    '{row['LEAD_TIME_STATED_TYPE']}',
                    '{row['LEAD_TIME_STATED_DAYS']}',
                    NULLIF('{row['MINIMUM_BKT_0']}', '')::numeric,
                    '{row['MINIMUM_BKT_0_TYPE']}',
                    NULLIF('{row['MAXIMUM_BKT']}', '')::numeric,
                    '{row['MAXIMUM_BKT_TYPE']}',
                    NULLIF('{row['MIN_BKT_2ND']}', '')::numeric,
                    '{row['MIN_BKT_TYPE_2ND']}',
                    NULLIF('{row['MAX_BKT_2ND']}', '')::numeric,
                    '{row['MAX_BKT_TYPE_2ND']}',
                    '{row['FIXED_REVIEW_MON']}',
                    '{row['FIXED_REVIEW_TUE']}',
                    '{row['FIXED_REVIEW_WED']}',
                    '{row['FIXED_REVIEW_THU']}',
                    '{row['FIXED_REVIEW_FRI']}',
                    '{row['FIXED_REVIEW_SAT']}',
                    '{row['FIXED_REVIEW_SUN']}',
                    NULLIF('{row['VENDOR_SERVICE_LEVEL']}', '')::numeric,
                    '{row['DAYS_TO_BUY_OVERRIDE']}',
                    '{row['ORDER_SEQUENCE']}',
                    '{row['LOAD_BLDG_LIMIT_FLG']}',
                    '{row['ORDER_FILTER_FLAG']}',
                    '{row['request_id']}',
                    current_timestamp,
                    current_timestamp,
                    '-1',
                    '-1'
                )
                ON CONFLICT (CS_VENDOR_ID) DO UPDATE SET
                    VENDOR_NUMBER = EXCLUDED.VENDOR_NUMBER,
                    CONNECTING_VENDOR = EXCLUDED.CONNECTING_VENDOR,
                    TRANSIT_DAYS = EXCLUDED.TRANSIT_DAYS,
                    LAG_DAYS = EXCLUDED.LAG_DAYS,
                    LEAD_TIME_STATED_TYPE = EXCLUDED.LEAD_TIME_STATED_TYPE,
                    LEAD_TIME_STATED_DAYS = EXCLUDED.LEAD_TIME_STATED_DAYS,
                    MINIMUM_BKT_0 = EXCLUDED.MINIMUM_BKT_0,
                    MINIMUM_BKT_0_TYPE = EXCLUDED.MINIMUM_BKT_0_TYPE,
                    MAXIMUM_BKT = EXCLUDED.MAXIMUM_BKT,
                    MAXIMUM_BKT_TYPE = EXCLUDED.MAXIMUM_BKT_TYPE,
                    MIN_BKT_2ND = EXCLUDED.MIN_BKT_2ND,
                    MIN_BKT_TYPE_2ND = EXCLUDED.MIN_BKT_TYPE_2ND,
                    MAX_BKT_2ND = EXCLUDED.MAX_BKT_2ND,
                    MAX_BKT_TYPE_2ND = EXCLUDED.MAX_BKT_TYPE_2ND,
                    FIXED_REVIEW_MON = EXCLUDED.FIXED_REVIEW_MON,
                    FIXED_REVIEW_TUE = EXCLUDED.FIXED_REVIEW_TUE,
                    FIXED_REVIEW_WED = EXCLUDED.FIXED_REVIEW_WED,
                    FIXED_REVIEW_THU = EXCLUDED.FIXED_REVIEW_THU,
                    FIXED_REVIEW_FRI = EXCLUDED.FIXED_REVIEW_FRI,
                    FIXED_REVIEW_SAT = EXCLUDED.FIXED_REVIEW_SAT,
                    FIXED_REVIEW_SUN = EXCLUDED.FIXED_REVIEW_SUN,
                    VENDOR_SERVICE_LEVEL = EXCLUDED.VENDOR_SERVICE_LEVEL,
                    DAYS_TO_BUY_OVERRIDE = EXCLUDED.DAYS_TO_BUY_OVERRIDE,
                    ORDER_SEQUENCE = EXCLUDED.ORDER_SEQUENCE,
                    LOAD_BLDG_LIMIT_FLG = EXCLUDED.LOAD_BLDG_LIMIT_FLG,
                    ORDER_FILTER_FLAG = EXCLUDED.ORDER_FILTER_FLAG,
                    REQUEST_ID = EXCLUDED.REQUEST_ID,
                    CREATION_DATE = EXCLUDED.CREATION_DATE,
                    LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
                    CREATED_BY = EXCLUDED.CREATED_BY,
                    LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY
                '''
            )



            cur.execute(
        f'''
        INSERT INTO XXPO_VENDOR_COSTING_INFO (
            CS_VENDOR_ID,
            FREIGHT_ALLOW_BUY_RATE,
            FREIGHT_ALLOW_BUY_RATE_TYPE,
            FREIGHT_ALLOW_BUY_RATE_CHANGED,

            PREPAYADD_BUY_RATE_TYPE,
            PREPAYADD_BUY_RATE_CHANGED,
            FREIGHT_BILL_BUY_RATE,
            FREIGHT_BILL_BUY_RATE_TYPE,
            FREIGHT_BILL_BUY_RATE_CHANGED,
            BACKHAUL_BUY_RATE,
            BACKHAUL_BUY_RATE_TYPE,
            BACKHAUL_BUY_RATE_CHANGED,
            PRE_PAY_FLAG,
            PRE_PAY_ADD_FLAG,
            FREIGHT_BILL_FLAG,
            BACKHAUL_FLAG,
            EXTERNAL_BACKHAUL_FLAG,
            SSA_RATE,
            SSA_RATE_TYPE,
            SSA_RATE_BASIS_NET_FLAG,
            SSA_RATE_BASIS_GROSS_FLAG,
            SSA_RATE_BASIS_ACC,
            CASH_DISC_TERMS,
            CASH_DISC_TERMS_DAYS,
            CASH_DISC_TERMS_NET,
            REQUEST_ID,
            CREATION_DATE,
            LAST_UPDATE_DATE,
            CREATED_BY,
            LAST_UPDATED_BY
        )
        VALUES (
            {generated_c1},
            NULLIF('{row['FREIGHT_ALLOW_BUY_RATE']}', '')::numeric,
            '{row['FREIGHT_ALLOW_BUY_RATE_TYPE']}',
            {f"NULLIF('{row['FREIGHT_ALLOW_BUY_RATE_CHANGED']}', '')::date" if row['FREIGHT_ALLOW_BUY_RATE_CHANGED'] else 'NULL'},

            '{row['PREPAYADD_BUY_RATE_TYPE']}',
            {f"NULLIF('{row['PREPAYADD_BUY_RATE_CHANGED']}', '')::date" if row['PREPAYADD_BUY_RATE_CHANGED'] else 'NULL'},
            NULLIF('{row['FREIGHT_BILL_BUY_RATE']}', '')::numeric,
            '{row['FREIGHT_BILL_BUY_RATE_TYPE']}',
            {f"NULLIF('{row['FREIGHT_BILL_BUY_RATE_CHANGED']}', '')::date" if row['FREIGHT_BILL_BUY_RATE_CHANGED'] else 'NULL'},
            NULLIF('{row['BACKHAUL_BUY_RATE']}', '')::numeric,
            '{row['BACKHAUL_BUY_RATE_TYPE']}',
            {f"NULLIF('{row['BACKHAUL_BUY_RATE_CHANGED']}', '')::date" if row['BACKHAUL_BUY_RATE_CHANGED'] else 'NULL'},
            '{row['PRE_PAY_FLAG']}',
            '{row['PRE_PAY_ADD_FLAG']}',
            '{row['FREIGHT_BILL_FLAG']}',
            '{row['BACKHAUL_FLAG']}',
            '{row['EXTERNAL_BACKHAUL_FLAG']}',
            NULLIF('{row['SSA_RATE']}', '')::numeric,
            '{row['SSA_RATE_TYPE']}',
            '{row['SSA_RATE_BASIS_NET_FLAG']}',
            '{row['SSA_RATE_BASIS_GROSS_FLAG']}',
            '{row['SSA_RATE_BASIS_ACC']}',
            '{row['CASH_DISC_TERMS']}',
            '{row['CASH_DISC_TERMS_DAYS']}',
            '{row['CASH_DISC_TERMS_NET']}',
            '{row['request_id']}',
            current_timestamp,
            current_timestamp,
            '-1',
            '-1'
        )
        ON CONFLICT (CS_VENDOR_ID) DO UPDATE SET
            FREIGHT_ALLOW_BUY_RATE = EXCLUDED.FREIGHT_ALLOW_BUY_RATE,
            FREIGHT_ALLOW_BUY_RATE_TYPE = EXCLUDED.FREIGHT_ALLOW_BUY_RATE_TYPE,
            FREIGHT_ALLOW_BUY_RATE_CHANGED = EXCLUDED.FREIGHT_ALLOW_BUY_RATE_CHANGED,
            PREPAYADD_BUY_RATE_TYPE = EXCLUDED.PREPAYADD_BUY_RATE_TYPE,
            PREPAYADD_BUY_RATE_CHANGED = EXCLUDED.PREPAYADD_BUY_RATE_CHANGED,
            FREIGHT_BILL_BUY_RATE = EXCLUDED.FREIGHT_BILL_BUY_RATE,
            FREIGHT_BILL_BUY_RATE_TYPE = EXCLUDED.FREIGHT_BILL_BUY_RATE_TYPE,
            FREIGHT_BILL_BUY_RATE_CHANGED = EXCLUDED.FREIGHT_BILL_BUY_RATE_CHANGED,
            BACKHAUL_BUY_RATE = EXCLUDED.BACKHAUL_BUY_RATE,
            BACKHAUL_BUY_RATE_TYPE = EXCLUDED.BACKHAUL_BUY_RATE_TYPE,
            BACKHAUL_BUY_RATE_CHANGED = EXCLUDED.BACKHAUL_BUY_RATE_CHANGED,
            PRE_PAY_FLAG = EXCLUDED.PRE_PAY_FLAG,
            PRE_PAY_ADD_FLAG = EXCLUDED.PRE_PAY_ADD_FLAG,
            FREIGHT_BILL_FLAG = EXCLUDED.FREIGHT_BILL_FLAG,
            BACKHAUL_FLAG = EXCLUDED.BACKHAUL_FLAG,
            EXTERNAL_BACKHAUL_FLAG = EXCLUDED.EXTERNAL_BACKHAUL_FLAG,
            SSA_RATE = EXCLUDED.SSA_RATE,
            SSA_RATE_TYPE = EXCLUDED.SSA_RATE_TYPE,
            SSA_RATE_BASIS_NET_FLAG = EXCLUDED.SSA_RATE_BASIS_NET_FLAG,
            SSA_RATE_BASIS_GROSS_FLAG = EXCLUDED.SSA_RATE_BASIS_GROSS_FLAG,
            SSA_RATE_BASIS_ACC = EXCLUDED.SSA_RATE_BASIS_ACC,
            CASH_DISC_TERMS = EXCLUDED.CASH_DISC_TERMS,
            CASH_DISC_TERMS_DAYS = EXCLUDED.CASH_DISC_TERMS_DAYS,
            CASH_DISC_TERMS_NET = EXCLUDED.CASH_DISC_TERMS_NET,
            REQUEST_ID = EXCLUDED.REQUEST_ID,
            CREATION_DATE = EXCLUDED.CREATION_DATE,
            LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
            CREATED_BY = EXCLUDED.CREATED_BY,
            LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY
        ''')


            cur.execute(f'''INSERT INTO XXPO_VENDOR_EDI_INFO (CS_VENDOR_ID,
                VENDOR_CONTINUITY_REPLEN_FLG,
                REQUEST_ID,
                CREATION_DATE,
                LAST_UPDATE_DATE,
                CREATED_BY,
                LAST_UPDATED_BY
                ) 
                VALUES (
                {generated_c1},
                '{row['VENDOR_CONTINUITY_REPLEN_FLG']}',
                '{row['request_id']}',
                current_timestamp,
                current_timestamp,
                '-1',
                '-1') 
                ON CONFLICT (CS_VENDOR_ID) 
                DO UPDATE SET VENDOR_CONTINUITY_REPLEN_FLG = EXCLUDED.VENDOR_CONTINUITY_REPLEN_FLG,
                REQUEST_ID = EXCLUDED.REQUEST_ID,
                CREATION_DATE = EXCLUDED.CREATION_DATE,
                LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
                CREATED_BY = EXCLUDED.CREATED_BY,
                LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY
                ''')
                
print(f'Inserted into all Vendor master and all 4 child tables')

cur.execute(
            """UPDATE  XXPO_ORCL2CSPOMS_VEND_STG
               SET PROCESS_STATUS = 'P'
               where PROCESS_STATUS ='N'
            """
        )



# Commit changes and close the connection
merged_df = merged_df.fillna('')
merged_df = merged_df.drop_duplicates(subset=['VENDOR_FACILITY', 'VENDOR_NUMBER'])

upsert_vendor_master_and_childs(merged_df,cur)

# conn.commit()
# cur.close()
# conn.close()

Request ID: 154
Current Timestamp: 2024-01-22 19:48:06
mapped_values:########0 {'BIC_VENDOR_NUMBER': '445932', 'BIC_BUYER_NUMBER': 'J8', 'BIC_CRP_LEVEL': '', 'BIC_STATUS': 'A', 'BIC_PO_RECOMMENDED_IND': 'B', 'BIC_AFE_TYPE1': '', 'BIC_AFE_FAX_NUMBER': '', 'BIC_AFE_CONTACT': '', 'BIC_MINIMUM_QUANTITY': '0000008', 'BIC_MINIMUM_TYPE': 'P', 'BIC_BRACKET_QUANTITY': '0000001', 'BIC_CURRENT_BKT_NUMBER': '0', 'BIC_CURRENT_BKT_QUANTITY': '0000026', 'BIC_CURRENT_BKT_TYPE': 'P', 'BIC_MAXIMUM_QUANTITY': '0000026', 'BIC_MAXIMUM_TYPE': 'P', 'BIC_ORDER_INTERVAL_WEEKS': '000.00', 'BIC_LEAD_TIME_STATED_WEEKS': '002.01', 'BIC_FIXED_REVIEW_DAY1': '', 'BIC_FIXED_REVIEW_DAY2': '', 'BIC_FIXED_REVIEW_DAY3': '', 'BIC_FIXED_REVIEW_DAY4': '', 'BIC_FIXED_REVIEW_DAY5': '', 'BIC_FIXED_REVIEW_DAY6': '', 'BIC_FIXED_REVIEW_DAY7': '', 'BIC_TARGET_SERVICE_LEVEL': '0095', 'BIC_NAME': 'INTL. TRADE IMPORTS', 'BIC_ADDRESS_1': '253 CASTLEBERRY INDUSTRIAL DR', 'BIC_CITY': 'CUMMINGS', 'BIC_STATE': 'GA', 'BIC_ZIP': '30040', 'BI

INSERT INTO CSPOMS.XXPO_ORCL2CSPOMS_VEND_STG (BIC_VENDOR_NUMBER,BIC_BUYER_NUMBER,BIC_CRP_LEVEL,BIC_STATUS,BIC_PO_RECOMMENDED_IND,BIC_AFE_TYPE1,BIC_AFE_FAX_NUMBER,BIC_AFE_CONTACT,BIC_MINIMUM_QUANTITY,BIC_MINIMUM_TYPE,BIC_BRACKET_QUANTITY,BIC_CURRENT_BKT_NUMBER,BIC_CURRENT_BKT_QUANTITY,BIC_CURRENT_BKT_TYPE,BIC_MAXIMUM_QUANTITY,BIC_MAXIMUM_TYPE,BIC_ORDER_INTERVAL_WEEKS,BIC_LEAD_TIME_STATED_WEEKS,BIC_FIXED_REVIEW_DAY1,BIC_FIXED_REVIEW_DAY2,BIC_FIXED_REVIEW_DAY3,BIC_FIXED_REVIEW_DAY4,BIC_FIXED_REVIEW_DAY5,BIC_FIXED_REVIEW_DAY6,BIC_FIXED_REVIEW_DAY7,BIC_TARGET_SERVICE_LEVEL,BIC_NAME,BIC_ADDRESS_1,BIC_CITY,BIC_STATE,BIC_ZIP,BIC_PHONE,BIC_CONTACT,BIC_TRANSFER_MATCH_TYPE,BIC_PAYABLE_VENDOR_NBR,BIC_LEAD_TIME_WEEKS_TYPE,BIC_DUNS_REMIT_NO,BIC_DUNS_REMIT_SUFFIX,BIC_FACILITY_SHIP_TO,BIC_PRE_SCHED_FLAG,BIC_BROKER_NUMBER,BIC_FREIGHT_ALLOW,BIC_FREIGHT_ALLOW_TYPE,BIC_TERMS_PERCENT,BIC_TERMS_DAYS,BIC_TERMS_NET_DAYS,BIC_TERMS_BASE,BIC_BILL_FOR_SHORT_DEALS_FLAG,BIC_TRANSIT_DAYS,BIC_DUNS_NO,BIC_2ND_MIN_QUAN

@@@@@@@@@ result_df Empty DataFrame
Columns: []
Index: []
@@@@@@@@@ merged_df    stg_seq_id  VENDOR_FACILITY bic_vendor_number bic_buyer_number  \
0          11              251            445932               J8   
1          12              252            447709               J8   

  bic_crp_level bic_status bic_po_recommended_ind bic_afe_type1  \
0                        A                      B                 
1                        A                      B                 

  PO_TRANSMISSION_VALUE PO_TRANSMISSION_TYPE  ... LOAD_BUILDING  \
0                                             ...             B   
1                                             ...             B   

             FACILITY_NAME PREPAYADD_BUY_RATE_CHANGED  \
0  C&S GS EXE FROZEN - PER                 2022-07-20   
1         C&S GS RR GM-HBC                 2022-08-05   

  BACKHAUL_BUY_RATE_CHANGED FREIGHT_BILL_BUY_RATE_CHANGED  \
0                2022-07-20                    2022-07-20   
1               

In [2]:
final_df['CONNECTING_VENDOR']

0    None
1    None
Name: CONNECTING_VENDOR, dtype: object